<a href="https://colab.research.google.com/github/JumperWacky/Jumper/blob/main/mnist_rec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset

In [4]:
df = pandas.read_csv("/content/mnist_data/mnist_test.csv")
row = 0
data = df.iloc[row]

label = data[0]

img = data[1:].values.reshape(28, 28)
plt.title("label = " + str(label))
plt.imshow(img, interpolation = 'none', cmap = 'Blues')
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/content/mnist_data/mnist_test.csv'

In [6]:
class Classifier(nn.Module):
  def __init__(self):
    # 初始化Pytorch父类
    super(Classifier).__init__()

    # 定义神经网络层
    self.model = nn.Sequential(
        nn.Linear(784, 200),
        nn.Sigmoid(),
        nn.Linear(200, 10),
        nn.Sigmoid()
    )

    # 创建损失函数
    self.loss_function = nn.MSELoss()

    # 创建优化器
    self.optimizer = torch.optim.SGD(self.parameters(), lr=0.01)

    # 可视化训练，记录训练进展的计数器和列表
    self.counter = 0
    self.progress = []

    pass
  
  def forward(self, inputs):
    return self.model(inputs)

  def train(self, inputs, targets):
    # 计算网络的输出值
    outputs = self.forward(inputs)
    # 计算损失值
    loss = self.loss_function(outputs, targets)

    # 梯度归零，反向传播，更新权重
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()

    # 每隔10个训练样本增加一次计数器的值，并将损失值添加进列表的末尾 
    self.counter += 1
    if (self.counter % 10 == 0):
      self.progress.append(loss.item())
      pass
    
    if (self.counter % 10000 == 0):
      print("counter = ", self.counter())
      pass
    
  def plot_progress(self):
    df = pandas.DataFrame(self.progress, columns=['loss'])
    df.plot(ylim=(0, 1.0), figsize=(16,8), alpha=0.1, marker=".", grid=True, yticks=(0, 0.25, 0.5))
    pass

SyntaxError: incomplete input (787938450.py, line 50)

In [ ]:
class MnistDataset(Dataset):
    def __init__(self, csv_file):
        self.data_df = pandas.read_csv(csv_file, header=None)
        pass
    
    def __len__():
        # Dataset类重写len()
        pass
    
    def __getitem__():
        # Dataset类重写getitem, 可通过下标访问
        pass